# Streamlit Model Preparation
In this notebook we will be creating more tailored models for each of our four cities. We will be using a [free weather api](https://www.weatherapi.com/) that will let us get current weather data for any gps coordinates that we choose and we will be translating this to weather data that our model can use.

In [18]:
import requests
import pandas as pd
import numpy as np
import pickle
import datetime as dt
from dateutil import parser
import folium
from branca.element import Figure

import xgboost as xg
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import plot_confusion_matrix, classification_report

In [19]:
pd.set_option("display.max_columns", None)
atl_data = pd.read_csv('../data/atl_df_cleaned.csv')
bos_data = pd.read_csv('../data/bos_df_cleaned.csv')
chi_data = pd.read_csv('../data/chi_df_cleaned.csv')
dia_data = pd.read_csv('../data/dia_df_cleaned.csv')
chi_data.head()

,severity,start_lat,start_lng,end_lat,end_lng,distance(mi),temperature(f),wind_chill(f),humidity(%),pressure(in),visibility(mi),wind_speed(mph),precipitation(in),amenity,bump,crossing,give_way,junction,no_exit,railway,roundabout,station,stop,traffic_calming,traffic_signal,turning_loop,sunrise_sunset,civil_twilight,nautical_twilight,astronomical_twilight,start_time_ep,end_time_ep,weather_timestamp_ep,year,month,week,accident_duration,side_L,side_R,wind_direction_CALM,wind_direction_E,wind_direction_ENE,wind_direction_ESE,wind_direction_N,wind_direction_NE,wind_direction_NNE,wind_direction_NNW,wind_direction_NW,wind_direction_S,wind_direction_SE,wind_direction_SSE,wind_direction_SSW,wind_direction_SW,wind_direction_VAR,wind_direction_W,wind_direction_WNW,wind_direction_WSW,weather_condition_Clear,weather_condition_Cloudy,weather_condition_Fog,weather_condition_Freezing Rain,weather_condition_Heavy Rain,weather_condition_Heavy Snow,weather_condition_Light Drizzle,weather_condition_Light Freezing Drizzle,weather_condition_Light Freezing Fog,weather_condition_Light Freezing Rain,weather_condition_Light Ice Pellets,weather_condition_Light Rain,weather_condition_Light Rain Showers,weather_condition_Light Rain with Thunder,weather_condition_Light Snow,weather_condition_Light Thunderstorms and Snow,weather_condition_Mist,weather_condition_Overcast,weather_condition_Partly Cloudy,weather_condition_Rain,weather_condition_Snow,weather_condition_Thunder,weather_condition_Thunderstorms and Rain,day_Friday,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday,hour_00,hour_01,hour_02,hour_03,hour_04,hour_05,hour_06,hour_07,hour_08,hour_09,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,weather_condition_Light Snow and Sleet
0,2,42.305960,-87.960150,42.306916,-87.960918,0.077,37.0,31.0,79.0,29.59,10.0,8.1,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0.0,0.0,1.480517e+09,1.480539e+09,1.480518e+09,2016,11,48,21600.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,2,42.175870,-88.135770,42.174620,-88.135794,0.086,37.0,28.1,76.0,29.62,10.0,15.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0.0,0.0,1.480518e+09,1.480540e+09,1.480518e+09,2016,11,48,21600.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,3,42.175897,-88.135769,42.174612,-88.135794,0.089,37.0,28.1,76.0,29.62,10.0,15.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0.0,0.0,1.480519e+09,1.480541e+09,1.480518e+09,2016,11,48,22524.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,3,41.860591,-87.992749,41.860590,-87.992700,0.003,37.0,30.4,76.0,29.61,10.0,9.2,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0.0,0.0,1.480522e+09,1.480544e+09,1.480522e+09,2016,11,48,21600.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,3,41.750330,-87.663440,41.750330,-87.663440,0.000,37.9,30.0,73.0,29.65,10.0,12.7,0.0,0,0,0,0,0,0,0,0,1,0,0,1,0,1.0,0.0,0.0,0.0,1.480524e+09,1.480545e+09,1.480524e+09,2016,11,48,21600.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [17]:
atl_data.head()

,severity,start_lat,start_lng,end_lat,end_lng,distance(mi),temperature(f),wind_chill(f),humidity(%),pressure(in),visibility(mi),wind_speed(mph),precipitation(in),amenity,bump,crossing,give_way,junction,no_exit,railway,roundabout,station,stop,traffic_calming,traffic_signal,turning_loop,start_time_ep,end_time_ep,weather_timestamp_ep,year,month,week,accident_duration,side_L,side_R,wind_direction_CALM,wind_direction_E,wind_direction_ENE,wind_direction_ESE,wind_direction_N,wind_direction_NE,wind_direction_NNE,wind_direction_NNW,wind_direction_NW,wind_direction_S,wind_direction_SE,wind_direction_SSE,wind_direction_SSW,wind_direction_SW,wind_direction_VAR,wind_direction_W,wind_direction_WNW,wind_direction_WSW,weather_condition_Clear,weather_condition_Cloudy,weather_condition_Fog,weather_condition_Heavy Rain,weather_condition_Light Drizzle,weather_condition_Light Freezing Rain,weather_condition_Light Ice Pellets,weather_condition_Light Rain,weather_condition_Light Rain Showers,weather_condition_Light Rain with Thunder,weather_condition_Light Snow,weather_condition_Mist,weather_condition_Overcast,weather_condition_Partly Cloudy,weather_condition_Rain,weather_condition_Snow,weather_condition_Thunder,weather_condition_Thunderstorms and Rain,sunrise_sunset_Day,sunrise_sunset_Night,sunrise_sunset_nan,civil_twilight_Day,civil_twilight_Night,civil_twilight_nan,nautical_twilight_Day,nautical_twilight_Night,nautical_twilight_nan,astronomical_twilight_Day,astronomical_twilight_Night,astronomical_twilight_nan,day_Friday,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday,hour_00,hour_01,hour_02,hour_03,hour_04,hour_05,hour_06,hour_07,hour_08,hour_09,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,weather_condition_Light Freezing Drizzle,weather_condition_Heavy Snow,weather_condition_Light Thunderstorms and Snow,weather_condition_Freezing Rain,weather_condition_Light Freezing Fog,weather_condition_Light Snow and Sleet
0,3,32.83752,-83.64864,32.840424,-83.646307,0.242,66.0,66.0,90.0,29.79,4.0,9.2,0.24,0,0,0,0,1,0,0,0,0,0,0,0,0,1.480527e+09,1.480549e+09,1.480528e+09,2016,11,48,21600.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,2,33.74153,-84.41434,33.746580,-84.430340,0.983,62.6,62.6,100.0,29.80,8.0,9.2,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,1.480528e+09,1.480549e+09,1.480528e+09,2016,11,48,21600.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,2,33.91220,-84.38390,33.911380,-84.372580,0.652,63.0,63.0,90.0,29.80,10.0,9.2,0.00,0,0,0,0,1,0,0,0,0,0,0,0,0,1.480529e+09,1.480551e+09,1.480529e+09,2016,11,48,21600.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,2,33.92736,-84.35774,33.986950,-84.339400,4.249,61.0,61.0,87.0,29.78,10.0,4.6,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,1.480531e+09,1.480553e+09,1.480532e+09,2016,11,48,21600.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,2,33.74136,-84.41402,33.741190,-84.411730,0.132,62.1,62.1,93.0,29.81,10.0,8.1,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,1.480532e+09,1.480554e+09,1.480532e+09,2016,11,48,21600.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


There are several weather columns that will not align with the data we will get in our api calls so we'll need to do a bit of condensing and reassigning. Since there is only ever one weather condition we can just take the sum of all the columns we want to combine and drop out the extra columns.

In [13]:
frame_dict = {
    'Atlanta': atl_data,
    'Boston' : bos_data,
    'Chicago': chi_data,
    'Denver' : dia_data
}
frame_dict.keys()

dict_keys(['Atlanta', 'Boston', 'Chicago', 'Denver'])

In [15]:
def build_models(df_dict):
    model_info = []
    for city in df_dict.keys():
        drop_features = ['severity','end_lat','end_lng','distance(mi)','amenity','bump','crossing','give_way','junction','no_exit','railway',
                         'roundabout','station','stop','traffic_calming','traffic_signal','turning_loop','end_time_ep','weather_timestamp_ep',
                         'accident_duration','side_R','sunrise_sunset','civil_twilight','nautical_twilight','astronomical_twilight']

        X = df_dict[city].drop(drop_features, axis=1)
        y = df_dict[city]['severity']
        # split
        X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)
        
        xg_model = xg.sklearn.XGBClassifier()
        xg_model.fit(X_train,y_train)
        
        with open(f'./models/{city}.pkl', mode='wb') as pickle_out:
            pickle.dump(xg_model, pickle_out)
        with open(f'./models/{city}_cols.pkl', mode='wb') as pickle_out:
            pickle.dump(X_train.columns, pickle_out)
        
        model_info.append(f'{city}: Training Accuracy: {xg_model.score(X_train,y_train)} Testing Accuracy: {xg_model.score(X_test,y_test)}')
        
    return model_info

In [16]:
build_models(frame_dict)

KeyError: "['sunrise_sunset' 'civil_twilight' 'nautical_twilight'\n 'astronomical_twilight'] not found in axis"

In [9]:
lat = 41.8781
lon = -87.6298
url = 'http://api.weatherapi.com/v1'
slug = '/current.json'
parameters = {
    'key': '0cb381c6bf3c4d4384b155640212911',
    'q': f'{lat},{lon}'
}

res = requests.get(url+slug, parameters)
weather_data = res.json()

In [10]:
sample_json = {
    'location': {
        'name': 'Chicago',
        'region': 'Illinois',
        'country': 'United States of America',
        'lat': 41.85,
        'lon': -87.65,
        'tz_id': 'America/Chicago',
        'localtime_epoch': 1638216524,
        'localtime': '2021-11-29 14:08'
    },
    'current': {
        'last_updated_epoch': 1638212400,
        'last_updated': '2021-11-29 13:00',
        'temp_c': 5.0,
        'temp_f': 41.0,
        'is_day': 1,
        'condition': {
            'text': 'Overcast',
            'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png',
            'code': 1009
        },
        'wind_mph': 11.9,
        'wind_kph': 19.1,
        'wind_degree': 190,
        'wind_dir': 'S',
        'pressure_mb': 1016.0,
        'pressure_in': 30.0,
        'precip_mm': 0.0,
        'precip_in': 0.0,
        'humidity': 36,
        'cloud': 100,
        'feelslike_c': 1.3,
        'feelslike_f': 34.4,
        'vis_km': 16.0,
        'vis_miles': 9.0,
        'uv': 1.0,
        'gust_mph': 13.9,
        'gust_kph': 22.3
    }
}

In [26]:
current_time = parser.parse(weather_data['location']['localtime'])
prediction_params = {
    'start_lat': lat,
    'start_lng': lon,
    'temperature(f)': weather_data['current']['temp_f'],
    'wind_chill(f)': weather_data['current']['feelslike_f'],
    'humidity(%)': weather_data['current']['humidity'],
    'pressure(in)': weather_data['current']['pressure_in'],
    'visibility(mi)': weather_data['current']['vis_miles'],
    'wind_speed(mph)': weather_data['current']['wind_mph'],
    'precipitation(in)': weather_data['current']['precip_in'],
    'year': current_time.strftime('%Y'),
    'month': current_time.strftime('%m'),
    'week': current_time.strftime('%U'),
    'start_time_ep': (current_time - dt.datetime(1970,1,1)).total_seconds(),
    f"wind_direction_{weather_data['current']['wind_dir']}": 1,
    f"day_{current_time.strftime('%A')}": 1,
    f"hour_{current_time.strftime('%H')}": 1
    
}
weather_string = f"{weather_data['current']['condition']['text']}\n{weather_data['current']['temp_f']} F ({weather_data['current']['feelslike_f']} F)\nHumidity: {weather_data['current']['humidity']}%\nPressure: {weather_data['current']['pressure_in']}\nVisibility: {weather_data['current']['vis_miles']} mi\nWind Spd/Dir: {weather_data['current']['wind_mph']} mph {weather_data['current']['wind_dir']}\nPrecipitation(in): {weather_data['current']['precip_in']}"

In [11]:
pred_df = pd.DataFrame(columns=X_train.columns)
pred_df = pred_df.append(prediction_params, ignore_index=True).fillna(0)
pred_df

,start_lat,start_lng,temperature(f),wind_chill(f),humidity(%),pressure(in),visibility(mi),wind_speed(mph),precipitation(in),year,month,week,start_time_ep,side_L,wind_direction_CALM,wind_direction_E,wind_direction_ENE,wind_direction_ESE,wind_direction_N,wind_direction_NE,wind_direction_NNE,wind_direction_NNW,wind_direction_NW,wind_direction_S,wind_direction_SE,wind_direction_SSE,wind_direction_SSW,wind_direction_SW,wind_direction_VAR,wind_direction_W,wind_direction_WNW,wind_direction_WSW,weather_condition_Blowing Snow,weather_condition_Clear,weather_condition_Cloudy,weather_condition_Fog,weather_condition_Freezing Rain,weather_condition_Heavy Rain,weather_condition_Heavy Snow,weather_condition_Light Drizzle,weather_condition_Light Freezing Drizzle,weather_condition_Light Freezing Fog,weather_condition_Light Freezing Rain,weather_condition_Light Ice Pellets,weather_condition_Light Rain,weather_condition_Light Rain with Thunder,weather_condition_Light Snow,weather_condition_Light Snow and Sleet / Windy,weather_condition_Light Thunderstorms and Snow,weather_condition_Mist,weather_condition_Mostly Cloudy / Windy,weather_condition_Overcast,weather_condition_Partly Cloudy,weather_condition_Rain,weather_condition_Snow,weather_condition_Thunder,weather_condition_Thunderstorms and Rain,day_Friday,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday,hour_00,hour_01,hour_02,hour_03,hour_04,hour_05,hour_06,hour_07,hour_08,hour_09,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,is_DST
0,41.8781,-87.6298,41.0,35.1,75,29.98,9.0,8.1,0.0,2021,12,48,1.638355e+09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
xg_model.predict(pred_df)

array([2], dtype=int64)

In [27]:
fig = Figure(width=550, height=350)
m1 = folium.Map(width=550, height=350, location=[lat,lon], zoom_start=10, max_zoom=12, min_zoom=8)
fig.add_child(m1)
folium.Marker(location=[lat,lon], popup='marker 1').add_to(m1)
folium.Marker(location=[lat,lon], popup=f'{weather_string}', ).add_to(m1)
m1

start_lat
start_lng
temperature(f)
wind_chill(f)
humidity(%)
pressure(in)
visibility(mi)
wind_speed(mph)
precipitation(in)
year
month
week
start_time_ep